In [34]:
from z3 import *

# Compute Min Hamming Distance

The function `compute_min_hamming_distance` calculates the minimum Hamming distance of a linear code defined by a generator matrix \( G \). The Hamming distance is the smallest number of differing bits between two codewords. Here's an intuitive breakdown of how this function works:

---

## **1. Initialize the Solver**
The function uses the Z3 SMT solver to handle logical constraints and solve for the minimum Hamming distance.
```python
solver = Solver()
```
This initializes the solver, which will be used to test various configurations of codewords.

---

## **2. Extract Dimensions of the Generator Matrix**
```python
k = len(G)      # Number of rows in G (basis vectors)
n = len(G[0])   # Length of each codeword
```
The generator matrix \( G \) has:
- \( k \): rows representing basis vectors.
- \( n \): columns representing the length of each codeword.

---

## **3. Define Boolean Variables for Row Selection**
```python
b = [Bool(f'b_{i}') for i in range(k)]
```
- Each `b[i]` is a Boolean variable.
- If `b[i] = True`, the corresponding row of \( G \) is selected to form a codeword.
- If `b[i] = False`, the row is not included.

---

## **4. Construct the Codeword**
```python
codeword = [
    Xor(*[b[i] if G[i][j] == 1 else False for i in range(k)])
    for j in range(n)
]
```
The codeword is computed bit-by-bit using the following logic:
1. For each bit \( j \) (from 0 to \( n-1 \)), check all rows of \( G \):
   - If the \( j \)-th element of the row is `1` and the row is selected (`b[i] = True`), include it in the XOR operation.
   - Otherwise, ignore it.
2. Use `Xor` to compute the binary sum of the selected rows modulo 2.

This ensures the codeword respects the binary addition rules of linear codes.

---

## **5. Print the Codeword Structure**
```python
print("Codeword structure (bitwise expressions):")
for j in range(n):
    print(f"codeword[{j}] =", codeword[j])
```
This step outputs the symbolic expressions for each bit in the codeword, allowing for debugging and verification.

---

## **6. Define the Hamming Weight Constraint**
```python
def hamming_weight_constraint(d_prime):
    hamming_weight = Sum([If(bit == True, 1, 0) for bit in codeword])
    print(f"Testing Hamming weight constraint for d' = {d_prime}: Hamming weight expression = {hamming_weight}")
    return hamming_weight == d_prime
```
- The Hamming weight is the number of `1`s in the codeword.
- For a given distance \( d' \), the function:
  - Sums `1` for each bit in the codeword that is `True`.
  - Adds a constraint that this sum equals \( d' \).

---

## **7. Perform Binary Search on \( d' \)**
The function performs a binary search over \( d' \), the tested Hamming weight:
1. **Set Bounds**: Start with `left = 1` and `right = max_distance`.
2. **Iterate**:
   - Test the midpoint \( d' \) using:
     ```python
     solver.add(hamming_weight_constraint(d_prime))
     ```
   - Check if a codeword with Hamming weight \( d' \) exists:
     - **SAT**: If a solution is found:
       - Record \( d' \) as a possible minimum distance.
       - Decrease `right` to test smaller values of \( d' \).
     - **UNSAT**: If no solution is found:
       - Increase `left` to test larger values of \( d' \).

---

## **8. Print and Evaluate Results**
When the solver finds a satisfiable result:
```python
model = solver.model()
computed_codeword = [model.evaluate(bit) for bit in codeword]
print("Computed codeword:", [1 if bit else 0 for bit in computed_codeword])
```
- The selected rows (`b[i]`) are displayed.
- The actual codeword values are computed and printed as a binary array.

---

## **9. Final Output**
The smallest \( d' \) for which a codeword exists is the minimum non-zero Hamming distance:
```python
if min_distance is not None:
    print(f"\nMinimum non-zero Hamming distance found: {min_distance}")
else:
    print("\nNo valid codeword found")
```

---

## **Example**
Given \( G = \):
```plaintext
[
    [1, 0, 0, 1, 1],
    [0, 1, 1, 0, 1],
    [1, 1, 0, 1, 0]
]
```
The function:
1. Constructs the symbolic codewords.
2. Tests for codewords with \( d' = 1, 2, 3, \dots \).
3. Outputs the minimum Hamming distance.

### Output:
```plaintext
Minimum Hamming distance: 3
```

This means the smallest non-zero codeword generated by \( G \) has exactly 3 bits set to `1`.

________
$
\text{Distance} = \min(\text{Hamming weight of XOR(Row \( i \), Row \( j \))})
$
where:
- \( i, j \) are indices of distinct rows in \( G \), and
- Hamming weight = the number of 1s in the resulting XOR.

### Steps to Calculate the Distance:

1. **Take All Combinations of Rows:**
   Compute the XOR of every pair of distinct rows \( i \) and \( j \) in the generator matrix \( G \).

2. **Count the Hamming Weight:**
   For each XOR result, count the number of 1s (non-zero bits). This is the Hamming weight of the resulting codeword.

3. **Find the Minimum Non-Zero Weight:**
   Among all the computed Hamming weights, the smallest non-zero weight is the **minimum Hamming distance** of the code.

---

### Why Do We Compare XORs of Rows?
The generator matrix \( G \) defines a **linear code** where the codewords are all possible linear combinations (XORs) of rows in \( G \). To find the minimum distance, you need to determine the smallest number of differing bits (Hamming weight) between any two distinct codewords, including those formed by XORing rows.

By construction:
- The XOR of two rows \( i \) and \( j \) produces another valid codeword.
- The Hamming weight of this XOR tells you how many bits differ between the two rows, effectively measuring their "distance."

In [35]:
from z3 import Solver, Bool, Xor, If, Or, sat
import numpy as np

In [36]:
def row_echelon_form(matrix):
    """
    Converts a binary matrix into row echelon form (REF).
    Assumes the matrix is in GF(2) (binary arithmetic: 0 and 1).
    """
    matrix = np.array(matrix, dtype=int)  # Ensure numpy integer array
    rows, cols = matrix.shape
    row_idx = 0

    for col_idx in range(cols):
        # Find a row with a leading 1 in the current column
        for i in range(row_idx, rows):
            if matrix[i, col_idx] == 1:
                # Swap rows if necessary
                matrix[[row_idx, i]] = matrix[[i, row_idx]]
                break
        else:
            # No pivot in this column; move to the next column
            continue

        # Eliminate all 1s below the pivot
        for i in range(row_idx + 1, rows):
            if matrix[i, col_idx] == 1:
                matrix[i] ^= matrix[row_idx]  # XOR the rows (binary subtraction)

        row_idx += 1
        if row_idx == rows:
            break

    return matrix.tolist()

In [42]:
from functools import reduce

In [66]:
# from functools import reduce
# from z3 import Solver, Bool, Xor, Sum, If, Or, sat

# def compute_min_hamming_distance(G, max_distance, use_ref=False):
#     # Optionally convert G to row echelon form
#     if use_ref:
#         print("Converting G to row echelon form...")
#         G = row_echelon_form(G)
#         print("G in REF:")
#         for row in G:
#             print(row)

#     # Initialize Z3 solver
#     solver = Solver()
    
#     # Number of rows in generator matrix G (number of codewords)
#     k = len(G)
#     n = len(G[0])  # Length of each codeword

#     # Define Boolean variables for each row of G
#     b = [Bool(f'b_{i}') for i in range(k)]
    
#     # Define codeword using Xor instead of Sum with modulo
#     def safe_xor(*args):
#         """Safely handle XOR for multiple arguments."""
#         if not args:  # No arguments
#             return False
#         if len(args) == 1:  # Single argument
#             return Xor(args[0], False)
#         # Recursively XOR all arguments
#         return reduce(lambda x, y: Xor(x, y), args)

#     # Generate the codeword structure
#     codeword = [
#         safe_xor(*[b[i] for i in range(k) if G[i][j] == 1])
#         for j in range(n)
#     ]

#     # Debug: Print the codeword structure for inspection
#     print("\nGenerated codeword structure:")
#     for j, expr in enumerate(codeword):
#         print(f"codeword[{j}] = {expr}")

#     # Define the Hamming weight constraint for a given tested distance d'
#     def hamming_weight_constraint(d_prime):
#         # Calculate Hamming weight of the codeword
#         hamming_weight = Sum([If(bit == True, 1, 0) for bit in codeword])
#         #print(f"\nAdding Hamming weight constraint: Hamming weight = {d_prime}")
#         return hamming_weight == d_prime

#     # Perform binary search on d' to find the minimum non-zero distance
#     left, right = 1, max_distance
#     min_distance = None

#     while left <= right:
#         d_prime = (left + right) // 2
#         solver.push()
        
#         # Add constraints for at least one non-zero b[i] and exact Hamming weight constraint
#         solver.add(Or(b))  # At least one b[i] should be True to avoid the all-zero codeword
#         solver.add(hamming_weight_constraint(d_prime))

#         print(f"\nChecking satisfiability for d' = {d_prime}")
#         if solver.check() == sat:
#             # Solution found with distance exactly equal to d_prime
#             print(f"\tSAT: Found a codeword with Hamming weight = {d_prime}")
#             model = solver.model()
            
#             # Print selected rows (b[i] values) in the model
#             print("\tSelected rows (b[i] values => True if found the codeword of value d' in this row):")
#             for i in range(k):
#                 print(f"\t\tb[{i}] = {model.evaluate(b[i])}")

#             # Compute and print the codeword values
#             computed_codeword = [model.evaluate(bit) for bit in codeword]
#             print("\tComputed codeword (binary):", [1 if bit else 0 for bit in computed_codeword])  # Convert boolean to binary

#             # Print detailed XOR operations
#             print("\n\tDetails of selected operations:")
#             selected_indices = [i for i in range(k) if model.evaluate(b[i])]
#             for idx in selected_indices:
#                 print(f"\t\tRow {idx}: {G[idx]}")

#             min_distance = d_prime
#             right = d_prime - 1  # Try for a smaller distance
#         else:
#             # No solution found with distance exactly equal to d_prime
#             print(f"\tUNSAT: No codeword with Hamming weight = {d_prime}")
#             left = d_prime + 1
        
#         solver.pop()

#     if min_distance is not None:
#         print(f"\nMinimum non-zero Hamming distance found: {min_distance}")
#     else:
#         print("\nNo valid codeword found")

#     return min_distance if min_distance is not None else "No valid codeword found"

# # Example generator matrix G and maximum distance for testing
# G = [
#         [1, 0, 1, 1, 0],
#         [0, 1, 1, 0, 1],
#         [1, 1, 0, 1, 1]
#     ]
# max_distance = 5  # Adjust as needed

# compute_min_hamming_distance(G, max_distance)



Generated codeword structure:
codeword[0] = Xor(b_0, b_2)
codeword[1] = Xor(b_1, b_2)
codeword[2] = Xor(b_0, b_1)
codeword[3] = Xor(b_0, b_2)
codeword[4] = Xor(b_1, b_2)

Checking satisfiability for d' = 3
	SAT: Found a codeword with Hamming weight = 3
	Selected rows (b[i] values => True if found the codeword of value d' in this row):
		b[0] = True
		b[1] = False
		b[2] = False
	Computed codeword (binary): [1, 0, 1, 1, 0]

	Details of selected operations:
		Row 0: [1, 0, 1, 1, 0]

Checking satisfiability for d' = 1
	UNSAT: No codeword with Hamming weight = 1

Checking satisfiability for d' = 2
	UNSAT: No codeword with Hamming weight = 2

Minimum non-zero Hamming distance found: 3


3

In [84]:
from functools import reduce
from z3 import Solver, Bool, Xor, Sum, If, Or, sat

def compute_min_hamming_distance(G, max_distance, use_ref=False):
    # Optionally convert G to row echelon form
    if use_ref:
        print("Converting G to row echelon form...")
        G = row_echelon_form(G)
        print("G in REF:")
        for row in G:
            print(row)

    # Initialize Z3 solver
    solver = Solver()
    
    # Number of rows in generator matrix G (number of codewords)
    k = len(G)
    n = len(G[0])  # Length of each codeword

    # Define Boolean variables for each row of G
    b = [Bool(f'b_{i}') for i in range(k)]
    
    # Define codeword using Xor instead of Sum with modulo
    def safe_xor(*args):
        """Safely handle XOR for multiple arguments."""
        if not args:  # No arguments
            return False
        if len(args) == 1:  # Single argument
            return Xor(args[0], False)
        # Recursively XOR all arguments
        return reduce(lambda x, y: Xor(x, y), args)

    # Generate the codeword structure
    codeword = [
        safe_xor(*[b[i] for i in range(k) if G[i][j] == 1])
        for j in range(n)
    ]

    # Debug: Print the codeword structure for inspection
    print("\nGenerated codeword structure:")
    for j, expr in enumerate(codeword):
        print(f"codeword[{j}] = {expr}")

    # Define the Hamming weight constraint for a given tested distance d'
    def hamming_weight_constraint(d_prime):
        # Calculate Hamming weight of the codeword
        hamming_weight = Sum([If(bit == True, 1, 0) for bit in codeword])
        #print(f"\nAdding Hamming weight constraint: Hamming weight = {d_prime}")
        return hamming_weight == d_prime

    # Perform binary search on d' to find the minimum non-zero distance
    left, right = 1, max_distance
    min_distance = None

    while left <= right:
        d_prime = (left + right) // 2
        solver.push()
        
        # Add constraints for at least one non-zero b[i] and exact Hamming weight constraint
        solver.add(Or(b))  # At least one b[i] should be True to avoid the all-zero codeword
        solver.add(hamming_weight_constraint(d_prime))

        print(f"\nChecking satisfiability for d' = {d_prime}")
        if solver.check() == sat:
            # Solution found with distance exactly equal to d_prime
            print(f"\tSAT: Found a codeword with Hamming weight = {d_prime}")
            model = solver.model()
            
            # Print selected rows (b[i] values) in the model
            # if b[i] == True, this row contributes to an operation for the satisfying codework
            selected_rows = []
            for i in range(k):
                if model.evaluate(b[i]):
                    selected_rows.append(i)
                #print(f"\t\tb[{i}] = {model.evaluate(b[i])}")

            # Compute and print the codeword values
            computed_codeword = [model.evaluate(bit) for bit in codeword]
            binary_codeword = [1 if bit else 0 for bit in computed_codeword]

            # # Print details of selected rows and XOR operations
            # print("\n\tDetails of how the codeword was generated:")
            # for col_idx, bit in enumerate(binary_codeword):
            #     contributing_rows = [
            #         i for i in selected_rows if G[i][col_idx] == 1
            #     ]
            #     print(f"\t\tcodeword[{col_idx}] = XOR of rows {contributing_rows} => {bit}")

            # Add the contributing rows
            print("\n\tContributing rows from G:")
            for idx in selected_rows:
                print(f"\t\tRow {idx}: {G[idx]}")
                
            print('\t\t      ________________________')
            print("XOR Result (codeword):", binary_codeword)


            min_distance = d_prime
            right = d_prime - 1  # Try for a smaller distance
        else:
            # No solution found with distance exactly equal to d_prime
            print(f"\tUNSAT: No codeword with Hamming weight = {d_prime}")
            left = d_prime + 1
        
        solver.pop()

    if min_distance is not None:
        print(f"\nMinimum non-zero Hamming distance found: {min_distance}")
    else:
        print("\nNo valid codeword found")

    return min_distance if min_distance is not None else "No valid codeword found"

In [83]:
from z3 import Solver, Bool, Xor, Sum, If, Or

# Define the pre-computed minimum Hamming distances for the matrices
precomputed_distances = {
    "G1": 3,  # Pre-computed Hamming distance for G1
    "G2": 2,  # Pre-computed Hamming distance for G2
    "G3": 1,  # Pre-computed Hamming distance for G3
    "G4": 5,  # Pre-computed Hamming distance for G4
    "G5": 2,  # Pre-computed Hamming distance for G5
    "G6": 3,  # Pre-computed Hamming distance for G6
    "G7": 2   # Pre-computed Hamming distance for G7
}

# List of matrices to test
matrices = {
    "G1": [
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 0],
        [1, 0, 1, 1, 1]
    ],
    "G2": [
        [1, 0, 1, 0, 1, 1, 0],
        [0, 1, 1, 1, 0, 0, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 1, 1]
    ],
    "G3": [
        [1, 0, 0, 1, 1],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1],
        [1, 1, 0, 1, 0],
        [0, 1, 1, 0, 1]
    ],
    "G4": [
        [1, 0, 1, 1, 0, 1, 1, 0],
        [0, 1, 0, 1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1, 1, 0, 1]
    ],
    "G5": [
        [1, 0, 1, 1, 0, 0, 1, 0, 1, 1],
        [0, 1, 0, 1, 1, 1, 0, 0, 1, 0],
        [1, 1, 0, 0, 1, 0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 1]
    ],
    "G6": [
        [1, 0, 1, 1, 0, 1],
        [0, 1, 1, 0, 1, 0]
    ],
    "G7": [
        [1, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 1, 0, 1, 0, 1, 1, 1],
        [1, 0, 1, 1, 1, 0, 0, 1, 1],
        [0, 1, 0, 1, 0, 1, 1, 0, 0]
    ]
}


# Iterate through matrices, compute distances, and verify results
for name, G in matrices.items():
    max_distance = len(G[0])  # Use the length of the codeword as the maximum possible distance
    print(f"Testing {name}...")
    computed_distance = compute_min_hamming_distance(G, max_distance)
    expected_distance = precomputed_distances[name]

    if computed_distance == expected_distance:
        print(f"PASS: {name} computed distance {computed_distance} matches expected {expected_distance}\n")
    else:
        print(f"FAIL: {name} computed distance {computed_distance} does not match expected {expected_distance}\n")
        
    print("___________________________________________________________________________")


Testing G1...

Generated codeword structure:
codeword[0] = Xor(b_0, b_2)
codeword[1] = Xor(b_0, b_1)
codeword[2] = Xor(b_1, b_2)
codeword[3] = Xor(b_1, b_2)
codeword[4] = Xor(b_0, b_2)

Checking satisfiability for d' = 3
	SAT: Found a codeword with Hamming weight = 3

	Contributing rows from G:
		Row 1: [0, 1, 1, 1, 0]
		      ________________________
XOR Result (codeword): [0, 1, 1, 1, 0]

Checking satisfiability for d' = 1
	UNSAT: No codeword with Hamming weight = 1

Checking satisfiability for d' = 2
	UNSAT: No codeword with Hamming weight = 2

Minimum non-zero Hamming distance found: 3
PASS: G1 computed distance 3 matches expected 3

___________________________________________________________________________
Testing G2...

Generated codeword structure:
codeword[0] = Xor(b_0, b_2)
codeword[1] = Xor(b_1, b_2)
codeword[2] = Xor(Xor(b_0, b_1), b_3)
codeword[3] = Xor(Xor(b_1, b_2), b_3)
codeword[4] = Xor(b_0, b_2)
codeword[5] = Xor(b_0, b_3)
codeword[6] = Xor(b_1, b_3)

Checking satisfi